In [32]:
import pickle
import pandas as pd
# ^^^ pyforest auto-imports - don't write above this line


# Imports

## Libraries

In [33]:
import unidecode
import wikipedia
import warnings
warnings.filterwarnings('ignore')
from functions import *

In [34]:
lst1 = [2,34]
if len(lst1)>= 2:
    print("hi")

hi


## Demonym Dictionary

In [35]:
with open("../my_saved_data/demonym_dictionary.pickle", "rb+") as f:
    demonym_dictionary = pickle.load(f)

# Adding City (Restaurant Inspection) Data
cities with most mexican immigrants in us: https://247wallst.com/economy/2017/01/27/us-cities-with-the-most-mexican-immigrants/

## NYC

In [36]:
rest_insp = pd.read_csv("../food_inspections_data/DOHMH_New_York_City_Restaurant_Inspection_Results.csv")

In [37]:
# tex-mex, mexican, latin
mex_rest_insp = rest_insp[rest_insp['CUISINE DESCRIPTION'] == 'Mexican']

In [38]:
mex_rest_insp.shape

(16656, 26)

In [39]:
mex_rest_insp.head(2)

,CAMIS,DBA,BORO,BUILDING,STREET,ZIPCODE,PHONE,CUISINE DESCRIPTION,INSPECTION DATE,ACTION,...,RECORD DATE,INSPECTION TYPE,Latitude,Longitude,Community Board,Council District,Census Tract,BIN,BBL,NTA
24,41571476,NATALIA BAR,Queens,62-10,39 AVENUE,11377.0,3477065942,Mexican,05/04/2018,Violations were cited in the following area(s).,...,06/14/2020,Cycle Inspection / Re-inspection,40.746844,-73.901406,402.0,26.0,26100.0,4028392.0,4.012320e+09,QN63
27,40996642,RECUERDOS MEXICANOS,Staten Island,232,PORT RICHMOND AVENUE,10302.0,7188155533,Mexican,10/22/2019,Violations were cited in the following area(s).,...,06/14/2020,Cycle Inspection / Initial Inspection,40.635784,-74.134935,501.0,49.0,20700.0,5025513.0,5.010820e+09,SI28


### Removing duplicate restaurants from nyc mexican restaurants df

In [40]:
unique_nyc_mex = mex_rest_insp.drop_duplicates(subset=['DBA'], keep='first')
unique_nyc_mex.reset_index(inplace=True, drop=True)
unique_nyc_mex.shape

(849, 26)

In [41]:
# list of restaurant names to compare to values in demonym_dictionary
nyc_rest_insp_names = [x.lower() for x in unique_nyc_mex['DBA']]

### Getting matches 

In [42]:
rest_matches_nyc = find_matches(nyc_rest_insp_names, demonym_dictionary)

In [43]:
len(rest_matches_nyc)

223

In [44]:
unique_rest_matches_nyc = set(rest_matches_nyc)

In [45]:
# was 44 before additions to demonym dict ... was 169 before word corrections
len(unique_rest_matches_nyc)

93

In [46]:
rest_matches_nyc[0:20]

['puebla seafood',
 'tacos puebla restaurant',
 'guadalajara de dia no.  2',
 'atlixco bakery & deli',
 'atlixco bakery & deli',
 'el tenampa restaurant',
 'el tenampa restaurant',
 'guadalupe inn',
 'guadalupe inn',
 'guadalupe inn',
 'guadalupe inn',
 'guadalupe inn',
 'guadalupe inn',
 'guadalupe inn',
 'guadalupe inn',
 'guadalupe inn',
 'guadalupe inn',
 'guadalupe inn',
 'guadalupe inn',
 'guadalupe inn']

### Getting Regions for Matches

In [ ]:
restaurant_matches_nyc = {key: None for key in set(unique_rest_matches_nyc)}

In [ ]:
for key, value in restaurant_matches_nyc.items():
    split_item = key.split()
    for word in split_item:
        for key1, value1 in demonym_dictionary.items():
            for city in value1:
                if word == city:
                    value_info = [key1, city]
                    restaurant_matches_nyc[key] = value_info

In [ ]:
len(restaurant_matches_nyc)

#### Removing all entries that passed filter incorrectly

In [ ]:
bad_keys = []
bad_matches = ['mexican','mexico','mexicanos','azteca','esperanza','estrada','esperanzas','salero','pinos',
                  'maria', 'bravo', 'nuevo', 'progreso', 'delicias', 'comales', 'palmas', 'palenque', 'concordia',
                 'china', 'gym', 'paloma', 'rio', 'mex', 'tequila', 'colorado', 'ventana', 'lom', 'garcia', 'paz',
                 'chavez', 'paraiso', 'senor', 'oriental', 'fronteras', 'tap', 'aca', 'purisima', 'rodriguez',
                 'hernandez', 'sanchez', 'victoria', 'oasis', 'cash', 'pinas', 'yaa',  'meson','agustin',
                 'agustin', 'limon', 'alamo', 'slp', 'providencia', 'reyes', 'lom', 'verde', 'perla', 'madrid', 
                 'delta', 'mama', 'lopez', 'honey', 'laurel', 'california pizza kitchen', 'sauces', 'laguna', 
              'dolores', 'presidio', 'ver', 'bernal', 'rincon', 'marin', 'palma', 'potrero', 'mid', 'valencia',
              'aura', 'kava', 'pueblito', 'castillo', 'tam', 'marcos', 'montecristo', 'tinajas', 'alvarado',
              'porvenir', 'nieves', 'mina', 'marin', "alamos", 'reforma', 'jal', 'margaritas', 'california']
for key, value in restaurant_matches_nyc.items():
    if value[1] in bad_matches:
        bad_keys.append(key)

In [ ]:
len(bad_keys)

In [ ]:
for x in bad_keys:
    if restaurant_matches_nyc.get(x) != None:
        del restaurant_matches_nyc[x]

In [ ]:
len(restaurant_matches_nyc)

In [ ]:
list(restaurant_matches_nyc.items())[:2]

### Getting coordinates for each restaurant

In [ ]:
region_in_name = []
for x in unique_nyc_mex['DBA']:
    if x.lower() in restaurant_matches_nyc.keys():
        region_in_name.append(1)
    else:
        region_in_name.append(0)

In [ ]:
unique_nyc_mex['region_in_name'] = region_in_name

In [ ]:
nyc_mex_rest_w_region = unique_nyc_mex[unique_nyc_mex['region_in_name'] == 1]
nyc_mex_rest_w_region.reset_index(inplace=True, drop=True)
nyc_mex_rest_w_region.shape

In [ ]:
nyc_mex_rest_w_region.DBA.nunique()

In [ ]:
nyc_mex_rest_w_region.head(2)

### Count of regions represented in nyc mexican restaurant names

In [ ]:
regions_represented = []
for key, value in restaurant_matches_nyc.items():
    if value[0] not in regions_represented:
        regions_represented.append(value[0])
regions_represented[0:3]

In [ ]:
region_list = []
for x in nyc_mex_rest_w_region['DBA']:
    region_list.append(restaurant_matches_nyc[x.lower()][0])
len(region_list)

In [ ]:
nyc_mex_rest_w_region['region'] = region_list

In [ ]:
# i get a lot of repeats with this method 
nyc_mex_rest_w_region['region'].value_counts()

### Getting region counts as a percentage

In [ ]:
nyc_region_counts_series = nyc_mex_rest_w_region['region'].value_counts()

In [ ]:
nyc_region_counts_dict = nyc_region_counts_series.to_dict()

In [ ]:
nyc_region_counts_dict_pct = {key: round(value/len(region_list), 2) 
                              for key, value in nyc_region_counts_dict.items()}

In [ ]:
list(nyc_region_counts_dict_pct.items())[0:2]

### Getting Values that matches

In [ ]:
nyc_values = {value[1]:0 for key, value in restaurant_matches_nyc.items()}

In [ ]:
for key, value in restaurant_matches_nyc.items():
    if value[1] in nyc_values.keys():
        nyc_values[value[1]] += 1

In [ ]:
# https://careerkarma.com/blog/python-sort-a-dictionary-by-value/
sort_values = sorted(nyc_values.items(), key=lambda x: x[1], reverse=True)
for i in sort_values[0:5]:
    print(i[0], i[1])

### Turning dictionary to df

In [ ]:
rest_df = pd.DataFrame.from_dict(restaurant_matches_nyc, orient="index")
rest_df.reset_index(inplace=True)
rest_df.columns = ['Restaurant_name', 'Region', 'Matching_word_w_region']

In [ ]:
rest_df.head()

In [ ]:
rest_df.columns

#### Checking to see if "Tulcingo Deli Grocery" is in database

In [ ]:
'TULCINGO' in list(nyc_mex_rest_w_region['DBA'].unique())

In [ ]:
tulcingo_count = 0
for x in rest_df.Matching_word_w_region:
    if x.lower() == 'tulcingo':
        tulcingo_count += 1
tulcingo_count

## Chicago 

In [ ]:
chi_data = pd.read_csv("./food_inspections_data/Food_Inspections_chicago.csv")
chi_data.shape

In [ ]:
chi_data.head(2)

In [ ]:
chi_rest = chi_data[chi_data['Facility Type'] == 'Restaurant']
chi_rest.shape

In [ ]:
# reduce the size of df by 33% 
chi_rest.shape[0] / chi_data.shape[0] 

In [ ]:
chi_rest.head(1)

### Getting Names of restaurants

In [ ]:
possible_chi_biz_names = list(zip(chi_rest['DBA Name'], chi_rest['AKA Name']))

In [ ]:
chi_biz_names = []
for x in possible_chi_biz_names:
    x_list = list(x)
    if x_list[0] == x_list[1]:
        del x_list[1]
    elif np.nan in x_list:
        x_list.remove(np.nan)
    lower_possible_chi_biz_names = [item.lower() for item in x_list]
    chi_biz_names.append(lower_possible_chi_biz_names)

In [ ]:
chi_biz_names[0:3]

In [ ]:
chi_rest['biz_names'] = chi_biz_names

### Getting Matches with Dem Dict

In [ ]:
rest_matches_chi = []
# list of not-quite-real matches
bad_matches = ['mexican','mexico','mexicanos','azteca','esperanza','estrada','esperanzas','salero','pinos',
                  'maria', 'bravo', 'nuevo', 'progreso', 'delicias', 'comales', 'palmas', 'palenque', 'concordia',
                 'china', 'gym', 'paloma', 'rio', 'mex', 'tequila', 'colorado', 'ventana', 'lom', 'garcia', 'paz',
                 'chavez', 'paraiso', 'senor', 'oriental', 'fronteras', 'tap', 'aca', 'purisima', 'rodriguez',
                 'hernandez', 'sanchez', 'victoria', 'oasis', 'cash', 'pinas', 'yaa', 'meson','agustin',
                 'agustin', 'limon', 'alamo', 'slp', 'providencia', 'reyes', 'lom', 'verde', 'perla', 'madrid', 
                 'delta', 'mama', 'lopez', 'honey', 'laurel', 'california pizza kitchen', 'sauces', 'laguna', 
              'dolores', 'presidio', 'ver', 'bernal', 'rincon', 'marin', 'palma', 'potrero', 'mid', 'valencia',
              'aura', 'kava', 'pueblito', 'castillo', 'tam', 'marcos', 'montecristo', 'tinajas', 'alvarado',
              'porvenir', 'nieves', 'mina', 'marin']
for item in chi_rest['biz_names']:
    for word in item:
        for key, value in demonym_dictionary.items():
            for city in value:
                if word == city:
                    if word not in bad_matches: # if not, the number of results is tripled
                        rest_matches_chi.append(item)

In [ ]:
len(rest_matches_chi)

In [ ]:
rest_matches_chi[0:5]

In [ ]:
unique_rest_matches_chi = []
for x in rest_matches_chi:
    if x not in unique_rest_matches_chi:
        unique_rest_matches_chi.append(x)
len(unique_rest_matches_chi)

In [ ]:
unique_rest_matches_chi[0:5]

### Getting Regions for Matches

In [ ]:
# using the DBA Name as the key
restaurant_matches_chi = {key[0]: None for key in unique_rest_matches_chi}

In [ ]:
for key, value in restaurant_matches_chi.items():
    split_item = key.split()
    for word in split_item:
        for key1, value1 in demonym_dictionary.items():
            for city in value1:
                if word == city:
                    value_info = [key1, city]
                    restaurant_matches_chi[key] = value_info
len(restaurant_matches_chi)

In [ ]:
for key, value in restaurant_matches_chi.items():
    if value == None:
        for x in unique_rest_matches_chi:
            if x[0] == key:
                if len(x) > 1:
                    split_item = [x.split() for x in x[1:]]
                    for word in split_item:
                        for key1, value1 in demonym_dictionary.items():
                            for city in value1:
                                if word == city:
                                    value_info = [key1, city]
                                    nyc_mex_rest_w_region[key] = value_info

In [ ]:
# not sure why some still equal None....
none_counter = 0
for key, value in restaurant_matches_chi.items():
    if value == None:
        none_counter += 1
none_counter

In [ ]:
# only 26 are genuine matches
len(restaurant_matches_chi) - none_counter

In [ ]:
# matching words
matching_words_list = []
for key, value in restaurant_matches_chi.items():
    if value != None:
        matching_words_list.append(value[1])

#### Removing values with a value of None

In [ ]:
bad_keys_chi = []
for key, value in restaurant_matches_chi.items():
    if value == None:
        bad_keys_chi.append(key)

#### Removing values that passed the filter incorrectly

In [ ]:
for key, value in restaurant_matches_chi.items():
    if value != None:
        if value[1] in bad_matches:
            bad_keys_chi.append(key)

In [ ]:
# removing bad keys
for x in bad_keys_chi:
    if restaurant_matches_chi.get(x) != None:
        del restaurant_matches_chi[x]
    elif restaurant_matches_chi.get(x) == None:
        del restaurant_matches_chi[x]

In [ ]:
len(restaurant_matches_chi.keys())

In [ ]:
restaurant_matches_chi

#### Correcting some errors in regions

In [ ]:
restaurant_matches_chi['ocotlan'][0] = 'Jalisco'

In [ ]:
restaurant_matches_chi['tecalitlan'][0] = "Jalisco"

In [ ]:
restaurant_matches_chi['uruapan'][0] = "Michoacan"

In [ ]:
restaurant_matches_chi

### Getting regions count

In [ ]:
regions_represented_chi = []
for key, value in restaurant_matches_chi.items():
    if value[0] not in regions_represented_chi:
        regions_represented_chi.append(value[0])

In [ ]:
regions_represented_chi[0:3]

In [ ]:
# getting the count of values
region_dict_chi = {key: 0 for key in regions_represented_chi}

In [ ]:
for key, value in restaurant_matches_chi.items():
    if value[0] in region_dict_chi.keys():
        region_dict_chi[value[0]] += 1

In [ ]:
region_dict_chi

In [ ]:
chicago_regions_represented = []
for key, value in region_dict_chi.items():
    item = [key, round(value/7, 2)]
    chicago_regions_represented.append(item)

In [ ]:
chicago_regions_represented

#### Making pct as a dict

In [ ]:
chicago_regions_pct = {value[0]:value[1] for value in chicago_regions_represented}

### Getting Values that matches

In [ ]:
chi_values = {value[1]:0 for key, value in restaurant_matches_chi.items()}

In [ ]:
for key, value in restaurant_matches_chi.items():
    if value[1] in chi_values.keys():
        chi_values[value[1]] += 1

In [ ]:
# https://careerkarma.com/blog/python-sort-a-dictionary-by-value/
sort_values_chi = sorted(chi_values.items(), key=lambda x: x[1], reverse=True)
for i in sort_values_chi[0:5]:
    print(i[0], i[1])

## Dallas

In [ ]:
dallas_data = pd.read_csv("./food_inspections_data/Dallas_inspections_October_2016_to_Present_.csv")
dallas_data.shape

In [ ]:
dallas_data.columns

### Only unique dallas restaurants

In [ ]:
unique_dallas_data = dallas_data.drop_duplicates(subset=['Restaurant Name'], keep='first')

In [ ]:
unique_dallas_data.shape

In [ ]:
# about an 84% reduction in size of df
unique_dallas_data.shape[0] / dallas_data.shape[0]

### Removing columns from data

In [ ]:
# most of the columns relate to inspections
col_names = list(unique_dallas_data.columns)

In [ ]:
col_names[0:11]

In [ ]:
col_names[-3:]
unique_dallas_data.drop(columns = [x for x in col_names[11:-4]], inplace=True)

In [ ]:
unique_dallas_data.head(2)

### Getting matches to dem dict

In [ ]:
# using the DBA Name as the key
restaurant_matches_dal = {key.lower(): None for key in unique_dallas_data['Restaurant Name'] if type(key) == str}

In [ ]:
for key, value in restaurant_matches_dal.items():
    if type(key) == str:
        split_item = key.split()
        for word in split_item:
            for key1, value1 in demonym_dictionary.items():
                for city in value1:
                    if word.lower() == city:
                        value_info = [key1, city]
                        restaurant_matches_dal[key] = value_info

In [ ]:
bad_keys_dal = []
for key, value in restaurant_matches_dal.items():
    if value == None:
        bad_keys_dal.append(key)
len(bad_keys_dal)

In [ ]:
len(restaurant_matches_dal)

In [ ]:
list(restaurant_matches_dal.items())[0:3]

### Removing restaurants with no matches

In [ ]:
for x in bad_keys_dal:
    if restaurant_matches_dal.get(x) != None:
        del restaurant_matches_dal[x]
    elif restaurant_matches_dal.get(x) == None:
        del restaurant_matches_dal[x]
len(list(restaurant_matches_dal.items()))

### Removing Bad Matches

In [ ]:
bad_keys_dal2 = []
for key, value in restaurant_matches_dal.items():
    if value != None:
        if value[1] in bad_matches:
            bad_keys_dal2.append(key)

In [ ]:
# removing bad keys
for x in bad_keys_dal2:
    if restaurant_matches_dal.get(x) != None:
        del restaurant_matches_dal[x]
    elif restaurant_matches_dal.get(x) == None:
        del restaurant_matches_dal[x]
len(restaurant_matches_dal)

### Getting the number of restaurants for each region

In [ ]:
regions_represented_dal = []
for key, value in restaurant_matches_dal.items():
    if value[0] not in regions_represented_dal:
        regions_represented_dal.append(value[0])

In [ ]:
# getting the count of values
region_dict_dal = {key: 0 for key in regions_represented_dal}
regions_represented_dal[0:4]

In [ ]:
for key, value in restaurant_matches_dal.items():
    if value[0] in region_dict_dal.keys():
        region_dict_dal[value[0]] += 1
region_dict_dal

### Getting Counts as percentages

In [ ]:
region_dict_dal_pct = {key:round(value/len(restaurant_matches_dal), 2) 
                       for key, value in region_dict_dal.items()}

### Getting Values that matches

In [ ]:
dal_values = {value[1]:0 for key, value in restaurant_matches_dal.items()}

In [ ]:
for key, value in restaurant_matches_dal.items():
    if value[1] in dal_values.keys():
        dal_values[value[1]] += 1

In [ ]:
# https://careerkarma.com/blog/python-sort-a-dictionary-by-value/
sort_values_dal = sorted(dal_values.items(), key=lambda x: x[1], reverse=True)
for i in sort_values_dal[0:5]:
    print(i[0], i[1])

## San Francisco 

In [ ]:
sf_data = pd.read_csv("./food_inspections_data/san_fran_Restaurant_Scores_-_LIVES_Standard.csv")
sf_data.shape

In [ ]:
sf_data.head(2)

In [ ]:
sf_data_unique = sf_data.drop_duplicates(subset=['business_name'], keep='first')
sf_data_unique.shape

In [ ]:
lower_biz_name = [x.lower() for x in sf_data['business_name']]
# this is slightly smaller than the set of unique business names without lowering all of the characters (by 1000)
len(set(lower_biz_name)) 

### Getting Names of Restaurants

In [ ]:
rest_matches_sf = []
for item in set(lower_biz_name):
    split_item = item.split()
    for word in split_item:
        for key, value in demonym_dictionary.items():
            for city in value:
                if word == city:
                    if word not in bad_matches: # if not, the number of results is tripled
                        rest_matches_sf.append(item)

In [ ]:
unique_sf_matches = list(set(rest_matches_sf))
len(unique_sf_matches)

In [ ]:
unique_sf_matches[0:3]

### Getting Regions for Matches

In [ ]:
restaurant_matches_sf = {key: None for key in set(unique_sf_matches)}

In [ ]:
for key, value in restaurant_matches_sf.items():
    split_item = key.split()
    for word in split_item:
        for key1, value1 in demonym_dictionary.items():
            for city in value1:
                if word == city:
                    value_info = [key1, city]
                    restaurant_matches_sf[key] = value_info
len(restaurant_matches_sf)

#### Removing all entries that passed filter incorrectly

In [ ]:
bad_keys = []
for key, value in restaurant_matches_nyc.items():
    if value[1] in bad_matches:
        bad_keys.append(key)
len(bad_keys)

In [ ]:
for x in bad_keys:
    if restaurant_matches_sf.get(x) != None:
        del restaurant_matches_sf[x]

In [ ]:
if restaurant_matches_sf.get('guadalupe elementary school'):
    del restaurant_matches_sf['guadalupe elementary school'] # probably refers to a neighborhood, not something in mex.

In [ ]:
len(restaurant_matches_sf)

### Getting Region Counts

In [ ]:
region_dict_sf = {value[0]: 0 for key, value in restaurant_matches_sf.items()}

In [ ]:
for key, value in restaurant_matches_sf.items():
    if value[0] in region_dict_sf.keys():
        region_dict_sf[value[0]] += 1

### Getting Region Counts as Percentages

In [ ]:
region_dict_sf_pct = {value[0]: 0 for key, value in restaurant_matches_sf.items()}

In [ ]:
for key, value in region_dict_sf.items():
    region_dict_sf_pct[key] = round(value/len(restaurant_matches_sf), 2)

In [ ]:
region_dict_sf_pct

### Getting Values that matches

In [ ]:
sf_values = {value[1]:0 for key, value in restaurant_matches_sf.items()}

In [ ]:
for key, value in restaurant_matches_sf.items():
    if value[1] in sf_values.keys():
        sf_values[value[1]] += 1

In [ ]:
# https://careerkarma.com/blog/python-sort-a-dictionary-by-value/
sort_values_sf = sorted(sf_values.items(), key=lambda x: x[1], reverse=True)
for i in sort_values_sf[0:5]:
    print(i[0], i[1])

# Combining All Restaurant Matches into One df

In [ ]:
# rest_df has all of the NYC results

## dict data

In [ ]:
chi_reg_w_pct = pd.DataFrame.from_dict(restaurant_matches_chi, orient='index', columns = ['Region', 'Match'])
chi_reg_w_pct.sort_index(axis=0, inplace=True)
chi_reg_w_pct['City'] = 'Chicago'

dal_reg_w_pct= pd.DataFrame.from_dict(restaurant_matches_dal, orient='index', columns = ['Region', 'Match'])
dal_reg_w_pct.sort_index(axis=0, inplace=True)
dal_reg_w_pct['City'] = 'Dallas'

sf_reg_w_pct= pd.DataFrame.from_dict(restaurant_matches_sf, orient='index', columns = ['Region', 'Match'])
sf_reg_w_pct.sort_index(axis=0, inplace=True)
sf_reg_w_pct['City'] = 'SF'

all_rest_df = pd.concat([chi_reg_w_pct, dal_reg_w_pct, sf_reg_w_pct], sort=True)

In [ ]:
all_rest_df.head(2)

## NYC data

In [ ]:
rest_df['City'] = 'NYC'

In [ ]:
rest_df.head()

In [ ]:
rest_df.set_index(keys=['Restaurant_name'], inplace=True)

In [ ]:
rest_df.columns = ['Region', 'Match', 'City']

In [ ]:
rest_df.columns

## Merging the two dfs

In [ ]:
all_restaurants = pd.concat([all_rest_df,rest_df])

## Saving df

In [ ]:
# all_restaurants.to_csv("./my_saved_data/Final_restaurant_list.csv")

# Using Wikipedia to correct any errors in data

In [ ]:
all_restaurants = pd.read_csv("./my_saved_data/Final_restaurant_list.csv", index_col = 0)
all_restaurants.head()

In [ ]:
all_regions = set(all_restaurants.Region)
all_regions = [x.lower() for x in all_regions]

In [ ]:
set_of_matches = set(all_restaurants.Match)
list_of_matches = list(set_of_matches)
len(list_of_matches)

## Getting summaries from wikipedia

In [ ]:
def count_num_of_entries(lst):
    counter = 0
    for y in lst:
        if type(y) != int:
            counter += 1
    print (counter)

In [ ]:
summaries = []
ambiguous_entries = []
for idx, match in enumerate(list_of_matches):
    try:
#         page = wikipedia.page(match)
        summary = wikipedia.summary(match)
        summaries.append(summary)
    except wikipedia.exceptions.DisambiguationError as e:
        if match + " (state)" in e.options:
#             page = wikipedia.page(match + " (state)")
            summary = wikipedia.summary(match + " (state)")
            summaries.append(summary)
        else:
            summaries.append(idx)
            ambiguous_entries.append((idx, e.options))

In [ ]:
# remove state names from list
no_states = []
for idx, x in enumerate(list_of_matches):
    if x in all_regions:
        no_states.append(x)
    else:
        no_states.append(idx)

In [ ]:
count_num_of_entries(no_states)

In [ ]:
# getting longest state name
max_state_name_len = 0
for x in all_regions:
    if len(x) > max_state_name_len:
        max_state_name_len = len(x)
max_state_name_len

In [ ]:
state_list = []
for idx, summary in enumerate(summaries):
    if type(no_states[idx]) == str:
        state_list.append(no_states[idx])
    elif type(summary) == str:
        if "tate of" in summary:
            beg_index = summary.index('tate of')
    #         try:
    #             end_index = summary.index(',', beg_index)
    #         except ValueError:
    #             continue
    #         try:
    #             end_index = summary.index('.', beg_index)
    #         except ValueError:
    #             continue
            state_name = summary[beg_index+8: beg_index+8+max_state_name_len]
            state_list.append(state_name)
        else:
            state_list.append(idx)
    else:
        state_list.append(idx)

In [ ]:
count_num_of_entries(state_list)

In [ ]:
name_list2 = []
for x in state_list:
    if type(x) == str:
        match = re.match(r"\w+", x)
        name = match.group()
        name_list2.append(name)
    else:
        name_list2.append(x)

In [ ]:
name_list3 = []
for idx, x in enumerate(name_list2):
    if type(x) == str:
        no_accent_x = unidecode.unidecode(x) 
        lower_x = no_accent_x.lower()
    else:
        lower_x = x
    single_name_regions = [x.split()[0] for x in all_regions]
    if lower_x in single_name_regions:
        name_list3.append(lower_x)
    else:
        name_list3.append(idx)

In [ ]:
for idx, x in enumerate(name_list3):
    if x == 'baja':
        print(idx)

In [ ]:
# after checking each case indivisually
name_list3[12] = 12
name_list3[82] = 'baja california'

In [ ]:
count_num_of_entries(name_list3)

## Going through articles not clearly associated with a region

In [ ]:
name_list3[0:5]

In [ ]:
# get options for all articles that aren't in name_list3
no_state_found_entries = []
for idx, x in enumerate(list_of_matches):
    if type(name_list3[idx]) == str:
        no_state_found_entries.append(name_list3[idx])
    else:
        options = wikipedia.search(x)
        no_state_found_entries.append((idx, x, options))

In [ ]:
final_list_of_region_matches = no_state_found_entries.copy()

In [ ]:
def show_next_nonmatch(lst=no_state_found_entries, num=0):
#     if type(lst[num][0]) == int:
    yield lst[num]
    yield from show_next_nonmatch(num=num+1)

In [ ]:
a = show_next_nonmatch()
a

In [ ]:
# all_regions

In [ ]:
next(a)

In [ ]:
final_list_of_region_matches[0] = "jalisco"
final_list_of_region_matches[1] = "hidalgo"
final_list_of_region_matches[9] = "nayarit"
# 12 is california... not sure about this one...nah, that's why it was appearing so much in sf
final_list_of_region_matches[15] = "michoacan"
final_list_of_region_matches[17] = "ciudad de mexico"
final_list_of_region_matches[20] = "puebla"
final_list_of_region_matches[27] = "mexico"
final_list_of_region_matches[28] = "hidalgo"
final_list_of_region_matches[32] = "jalisco"
final_list_of_region_matches[35] = "baja california"
final_list_of_region_matches[37] = "baja california"
final_list_of_region_matches[41] = "puebla"
final_list_of_region_matches[42] = "mexico"
final_list_of_region_matches[43] = "puebla"
final_list_of_region_matches[44] = "sinaloa"
final_list_of_region_matches[45] = "san luis potosi"
final_list_of_region_matches[53] = "puebla"
final_list_of_region_matches[55] = "chihuahua"
final_list_of_region_matches[56] = "zacatecas"
final_list_of_region_matches[57] = "coahuila"
final_list_of_region_matches[58] = "ciudad de mexico"
final_list_of_region_matches[59] = "guerrero"
final_list_of_region_matches[60] = "tamaulipas"
final_list_of_region_matches[61] = "guanajuato"
final_list_of_region_matches[63] = "zacatecas"
final_list_of_region_matches[64] = "sinaloa"
final_list_of_region_matches[65] = "chihuahua"
final_list_of_region_matches[69] = "veracruz"
final_list_of_region_matches[74] = "michoacan"
final_list_of_region_matches[79] = "ciudad de mexico"
final_list_of_region_matches[84] = "yucatan"
final_list_of_region_matches[85] = "nuevo leon"
final_list_of_region_matches[91] = "jalisco"
final_list_of_region_matches[97] = "veracruz"
final_list_of_region_matches[99] = "ciudad de mexico"
final_list_of_region_matches[103] = "hidalgo"
final_list_of_region_matches[104] = "michoacan"
final_list_of_region_matches[113] = "puebla"
final_list_of_region_matches[114] = "mexico"
final_list_of_region_matches[115] = "chihuahua"
final_list_of_region_matches[117] = "guanajuato"
final_list_of_region_matches[118] = "puebla"
final_list_of_region_matches[125] = "ciudad de mexico"
final_list_of_region_matches[127] = "sonora"

In [ ]:
counter = 0
for y in final_list_of_region_matches:
    if type(y) == str:
        counter += 1
print (counter)

## Fixing previous matches 

In [ ]:
final_list_of_matches_with_term = list(zip(final_list_of_region_matches, list_of_matches))

In [ ]:
final_list_of_matches_with_term_correct = [x for x in final_list_of_matches_with_term if type(x[0]) != tuple]

In [ ]:
matched_terms = [x[1] for x in final_list_of_matches_with_term_correct]

In [ ]:
match_boolean = []
new_region = []
for x in all_restaurants['Match']:
    if x in matched_terms:
        match_boolean.append(True)
        new_region_x = final_list_of_matches_with_term_correct[matched_terms.index(x)][0]
        new_region.append(new_region_x)
    else:
        match_boolean.append(False)

In [ ]:
sum(match_boolean)

In [ ]:
len(all_restaurants)

In [ ]:
len(new_region)

### Add match_boolean column and deleting on false condition

In [ ]:
all_restaurants['match_boolean'] = match_boolean

In [ ]:
all_correct_restaurants = all_restaurants[all_restaurants['match_boolean'] == True]

In [ ]:
all_correct_restaurants['new_region'] = new_region

In [ ]:
non_matches = 0
for num in range(len(all_correct_restaurants)):
    lower_orig_region = all_correct_restaurants['Region'][num].lower()
    if lower_orig_region != all_correct_restaurants['new_region'][num]:
        non_matches += 1

In [ ]:
non_matches # there were 98 incorrectly labelled matches

In [ ]:
len(new_region) - non_matches

In [ ]:
for idx, x in enumerate(all_correct_restaurants['new_region']):
    if x == 'nuevo':
        all_correct_restaurants['new_region'][idx] = 'nuevo leon'
    elif x == 'tepeaca':
        all_correct_restaurants['new_region'][idx] = 'puebla'
    elif x == 'quintana':
        all_correct_restaurants['new_region'][idx] = 'quintana roo'

In [ ]:
# set(all_correct_restaurants['new_region'])

## Saving all_correct_restaurants 

In [ ]:
# all_correct_restaurants.to_csv("./my_saved_data/all_correct_restaurants.csv")

# combining all pct results to a df 

- nyc_region_counts_dict_pct
- chicago_regions_pct
- region_dict_dal_pct
- region_dict_sf_pct

### Making them all have the same length

In [ ]:
regions_used = []

In [ ]:
def track_regions_found(dictionary, regions_used):
    for key, value in dictionary.items():
        if key not in regions_used:
            regions_used.append(key)
    return regions_used

In [ ]:
regions_used1 = track_regions_found(nyc_region_counts_dict_pct, regions_used)
regions_used2 = track_regions_found(chicago_regions_pct, regions_used1)
regions_used3 = track_regions_found(region_dict_dal_pct, regions_used2)
regions_used4 = track_regions_found(region_dict_sf_pct, regions_used3)

In [ ]:
# all but one region was found in these datasets (well 2 since michoacan was counted twice)
len(regions_used4)

In [ ]:
def add_keys_to_dict(dict_w_pct, regions_used = regions_used4):
    for region in regions_used:
        if region not in dict_w_pct.keys():
            dict_w_pct[region] = 0
    return dict_w_pct

In [ ]:
nyc_reg_w_pct = add_keys_to_dict(nyc_region_counts_dict_pct, regions_used4)
chi_reg_w_pct = add_keys_to_dict(chicago_regions_pct, regions_used4)
dal_reg_w_pct = add_keys_to_dict(region_dict_dal_pct)
sf_reg_w_pct = add_keys_to_dict(region_dict_sf_pct)

### Making into df

In [ ]:
nyc_reg_w_pct_df = pd.DataFrame.from_dict(nyc_reg_w_pct, orient='index', columns = ['NYC'])
nyc_reg_w_pct_df.sort_index(axis=0, inplace=True)

chi_reg_w_pct_df = pd.DataFrame.from_dict(chi_reg_w_pct, orient='index', columns = ['Chi'])
chi_reg_w_pct_df.sort_index(axis=0, inplace=True)

dal_reg_w_pct_df = pd.DataFrame.from_dict(dal_reg_w_pct, orient='index', columns = ['Dal'])
dal_reg_w_pct_df.sort_index(axis=0, inplace=True)

sf_reg_w_pct_df = pd.DataFrame.from_dict(sf_reg_w_pct, orient='index', columns = ['SF'])
sf_reg_w_pct_df.sort_index(axis=0, inplace=True)

In [ ]:
reg_rep_df = pd.concat([nyc_reg_w_pct_df, chi_reg_w_pct_df, dal_reg_w_pct_df, sf_reg_w_pct_df], sort=True, axis=1)

#### Fixing 2 Misnamed columns

In [ ]:
reg_rep_df.reset_index(inplace=True)

In [ ]:
reg_rep_df['Chi'][15] = reg_rep_df['Chi'][14]

In [ ]:
reg_rep_df.drop(index=14, inplace=True)

In [ ]:
reg_rep_df['Chi'][20] = reg_rep_df['Chi'][21]

In [ ]:
reg_rep_df.drop(index=21, inplace=True)

### Getting the count of the number of restaurants for each column

In [ ]:
rest_count_dict = {}
for col in reg_rep_df.columns[1:]:
    counter = 0
    for item in reg_rep_df[col]:
        if item > 0:
            counter += 1
    rest_count_dict[col] = counter   

In [ ]:
# despite all cities having more than 1.3 million people, 
# there is a vastly different number of mex. restaurants in each that have names referring to a city or region in mex.
rest_count_dict

#### Saving restaurant count dict

In [ ]:
# with open ('restaurant_counts.pickle', 'wb+') as f:
#     pickle.dump(rest_count_dict, f)

# Percent Matches for Each City

In [ ]:
# how many unique mexican restaurants were there for each city and how many were matched in the end

In [ ]:
num_nyc_restaurants = 0
for key, value in nyc_region_counts_dict.items():
    num_nyc_restaurants += value
num_nyc_restaurants

In [ ]:
num_nyc_restaurants / unique_nyc_mex.shape[0]
# chicago, dallas SF did not sort restaurants by cuisine type 

# total count df for all cities combined

In [ ]:
all_restaurants_cities_dict = {}
for dictionary in [nyc_region_counts_dict, region_dict_chi, region_dict_dal, region_dict_sf]:
    for key, value in dictionary.items():
        if key not in all_restaurants_cities_dict.keys():
            all_restaurants_cities_dict[key] = value
        elif key in all_restaurants_cities_dict.keys():
            all_restaurants_cities_dict[key] += value

In [ ]:
# moving michoacan to the key Michoacan de Ocampo
all_restaurants_cities_dict['Michoacan de Ocampo'] += all_restaurants_cities_dict['Michoacan']
del all_restaurants_cities_dict['Michoacan']

In [ ]:
all_rest_df = pd.DataFrame.from_dict(all_restaurants_cities_dict, orient='index', 
                                     columns = ['Count_of_restaurants'])

In [ ]:
all_rest_df.Count_of_restaurants.sum()

In [ ]:
all_rest_df['Percent_of_all_restaurants'] = round(all_rest_df['Count_of_restaurants'] / \
                                                  all_rest_df.Count_of_restaurants.sum(), 2)

In [ ]:
all_rest_df.sort_values(by = ['Percent_of_all_restaurants'], ascending=False).head()

In [ ]:
# all_rest_df.to_csv("./my_saved_data/all_restaurants_df.csv")

# Conclusions

All 4 cities have large Mexican immigrant populations (250k and above according to: https://247wallst.com/economy/2017/01/27/us-cities-with-the-most-mexican-immigrants/). Despite this surface similarity, it turns out that these immigrants do not all come from similar regions in Mexico. No region was in the top 5 most popular region for more than 2 of these cities. This suggests that immigration from Mexico was not dominated by one region but came from all regions of Mexico. Similarly, we see that there are no single region that makes up the majority of any US city's Mexican population: in all cases, no region accounted for more than 36% of the restaurants with regions associated with their names. This did have some variation, however: in San Francisco Guanajuato accounts for 40% of the restaurants while in Dallas the most dominant group accounts for 21%.

Some shortcomings of this project are the quality of this data: both for the demonym dictionary and the inspection data. The demonym dictionary had many entries, but there were some clear errors, e.g.: the restaurant 'jalisco' in Dallas was associated with "Ciudad de Mexico" instead of the region Jalisco. Additionally, it is not clear if **all** restaurants in each city are included in this data, or only restaurants that failed the inspection. In a similar vein, some cities had data that had too low of a quality to be useful (LA). Another problem with this project is deciding when a "match" is legitimate. For example, "Pueblito" ("little town") appeared often and is a town in Queretáro, but do these restaurants refer to the former or the latter? This was decided on an ad hoc basis and was founded solely on my (non-native) discretion. Lastly, the obvious issue of immediate representation being a marker of the true number of people from a certain region is certainly flawed. Just because there are say, a lot of Italian restaurants in a certain area, does not necessarily mean that there are many italian (recent) immigrants to that area. The popularity of food from certain regions may skew the data. 

Anecdotally, it appears that chain migration happens at a micro level. "Tulcingo" appears 5 times in the NYC data despite the fact that the town that it refers to in Mexico, Tulcingo del Valle, has only 9,000 people living in it (source: https://es.wikipedia.org/wiki/Municipio_de_Tulcingo)! 